# Most Suitable Zip Codes for Young Families Moving to the Boston Area

## 1. Import Packages and explore Massachusetts Metropolitan Area Shapefile

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt

In [ ]:
boundaries = gpd.read_file('data/MPO_Boundaries/MPO_Boundaries.shp')

In [ ]:
boundaries.head()

## 2. Import Zip Codes shapefile, change both layers to MA State Plane Projection

In [ ]:
zipcodes = gpd.read_file('data/zipcodes/tl_2010_25_zcta510.shp')

In [ ]:
zipcodes = zipcodes.to_crs("epsg:26986") #MA State Plane Projection

In [ ]:
boundaries = boundaries.to_crs("epsg:26986") 

In [ ]:
#Plot municipalities in MA for context

fig, ax = plt.subplots(figsize=(10, 10))
boundaries.plot(column='OBJECTID', cmap='PiYG', ax=ax).set_title('Massachusetts Municipal Boundaries', size=20)
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)
ax.set_axis_off()

In [ ]:
zipcodes.head()

In [ ]:
#Plot all zipcodes in MA for context

fig, ax = plt.subplots(figsize=(10, 10))
zipcodes.plot(column='ZCTA5CE10', cmap = 'spring', ax=ax)\
.set_title('Massachusetts Zip Codes', size=20)
cx.add_basemap(ax, source=cx.providers.Stamen.TonerLite, \
               crs=zipcodes.crs.to_string())
ax.set_axis_off()

## 3. Clip zip code layer to Boston Region MPO

In [ ]:
boston_region = boundaries[boundaries.MPO=="Boston Region"].copy()

In [ ]:
boston_region.plot()

In [ ]:
boston_zipcodes = gpd.clip(zipcodes, boston_region)

In [ ]:
boston_zipcodes.plot()

## 4. Analyze school availability per zip code

A variety of educational opportunities is important to many young families who wish to enroll their children in a high quality program. Below, I import school points, calculate the number of schools per zip code, and display the number of schools on a map.

In [ ]:
schools = gpd.read_file('data/schools/SCHOOLS_PT.shp')

In [ ]:
schools

In [ ]:
boston_zipcodes

In [ ]:
#Rename certain columns, and remove unecessary columns
boston_zipcodes.rename(columns={"ZCTA5CE10":"ZIPCODE"}, inplace = True)

In [ ]:
boston_zipcodes = boston_zipcodes[['ZIPCODE', 'ALAND10', 'geometry']]

In [ ]:
boston_zipcodes

In [ ]:
#Calculate the number of schools per zipcode
schools_count = schools[['SCHID', 'ZIPCODE']]\
.groupby('ZIPCODE').nunique().reset_index()

In [ ]:
schools_count.rename(columns = {"SCHID" : "school_count"}, inplace = True)

In [ ]:
schools_count

In [ ]:
# Add the school count column to "master" dataframe, boston_zipcodes
boston_zipcodes = boston_zipcodes.merge(schools_count, how='left', on='ZIPCODE')

In [ ]:
boston_zipcodes = boston_zipcodes.replace(np.nan,0)

In [ ]:
boston_zipcodes

In [ ]:
# Visualize the zipcodes with highest school count in a darker color
fig, ax = plt.subplots(figsize=(10, 10))
boston_zipcodes.plot(legend=True, column='school_count', \
                     cmap='PuRd', ax=ax, legend_kwds={'label': 'Number of Schools'})\
.set_title('Boston Area Schools by Zip Code', size = 20)
cx.add_basemap(ax, source=cx.providers.Stamen.TonerLite, \
               crs=boston_zipcodes.crs.to_string())
ax.set_axis_off()

In [ ]:
# List top zipcodes for number of schools
head_schools = boston_zipcodes.sort_values(by='school_count', ascending=False)
head_schools.head()

In [ ]:
# List zip codes with no schools
head_schools.tail()

If school quantity was the only consideration, the zip codes near Marlborough, Braintree, and Salem would be most desirable. However, families typically consider a number of factors before making a move. 

## 5. Library Availability 

The presence of libraries is also a valuable consideration for families with young readers. Similar to the above method with school points, Boston-area libraries were totaled for each zip code, and displayed on a map. 

In [ ]:
libraries = gpd.read_file('data/libraries/LIBRARIES_PT.shp')

In [ ]:
libraries

In [ ]:
# Calculate number of libraries per zip code
lib_count = libraries[['NAME', 'ZIP']].groupby('ZIP').nunique().reset_index()

In [ ]:
lib_count

In [ ]:
lib_count.rename(columns={"ZIP": "ZIPCODE"}, inplace=True)

In [ ]:
lib_count.rename(columns = {"NAME": "lib_count"}, inplace=True)

In [ ]:
lib_count

In [ ]:
boston_zipcodes = boston_zipcodes.merge(lib_count, how='left', on='ZIPCODE')

In [ ]:
boston_zipcodes

In [ ]:
boston_zipcodes = boston_zipcodes.replace(np.nan,0)

In [ ]:
#Plot number of libraries per zip code- darker color indicates more libraries
fig, ax = plt.subplots(figsize=(10, 10))
boston_zipcodes.plot(legend=True,column='lib_count', \
                     cmap='YlGnBu', ax=ax, \
                     legend_kwds={'label':'Number of Libraries'})\
.set_title('Boston Area Libraries by Zip Code', size=20)
cx.add_basemap(ax, source=cx.providers.Stamen.TonerLite, \
               crs=boston_zipcodes.crs.to_string())
ax.set_axis_off()

In [ ]:
# Zip codes with the highest number of libraries
head_lib = boston_zipcodes.sort_values(by='lib_count', ascending=False)
head_lib.head()

In [ ]:
# Zip codes with least amount of libraries
head_lib.tail()

While several zip codes appear to have at least two libraries within their boundaries, the zip codes near Lynn, Somerville, and Boston proper have a whopping 3 for families with book worms. 

## 6. Presence of Farmers Markets

Buying produce from local farmers markets is not only a fun weekend activity for families, but is a healthy and affordable nutrition option. The same analysis used for schools and libraries is also applied to farmers markets below. 

In [ ]:
farmers_markets = gpd.read_file('data/farmers_markets/FARMERSMARKETS_PT.shp')

In [ ]:
farmers_markets

In [ ]:
# Calculate number of farmers markets in each zip code
market_count = farmers_markets[['MARKET_ID', 'ZIP_CODE']]\
.groupby('ZIP_CODE').nunique().reset_index()

In [ ]:
market_count

In [ ]:
market_count.rename(columns = {"ZIP_CODE" : "ZIPCODE"}, inplace = True)

In [ ]:
market_count.rename(columns = {"MARKET_ID" : "market_count"}, inplace = True)

In [ ]:
boston_zipcodes = boston_zipcodes.merge(market_count, how='left', on='ZIPCODE')

In [ ]:
boston_zipcodes

In [ ]:
boston_zipcodes= boston_zipcodes.replace(np.nan, 0)

In [ ]:
#Plot the number of farmers markets per zip code
fig, ax = plt.subplots(figsize=(10, 10))
boston_zipcodes.plot(legend=True, column='market_count', cmap='YlOrRd', ax=ax, \
                     legend_kwds={'label':'Number of Farmers Markets'})\
.set_title('Boston Area Farmers Markets by Zip Code', size=20)
cx.add_basemap(ax, source=cx.providers.Stamen.TonerLite, \
               crs=boston_zipcodes.crs.to_string())
ax.set_axis_off()

In [ ]:
#Zip codes with the greatest number of farmers markets
head_markets = boston_zipcodes.sort_values(by='market_count', ascending=False)
head_markets.head()

In [1]:
#Zip codes with the least amount of farmers markets
head_markets.tail()

NameError: name 'head_markets' is not defined

As far as local produce goes, the nearer to Boston is the place to be. 

## 7. Ethnic Diversity

Diversity is an extremely important factor in a child's education and upbringing. If children make friends with others from different races, backgrounds, and ethnicities, they are more likely to have more empathy towards others as they grow up. An indicator of ethnic diversity is the percent population that speaks a language other than English. 

In [ ]:
language = pd.read_csv('data/language/language.csv', header=1)
language

In [ ]:
language.rename(columns = {'Estimate!!Total!!Population 5 years and over!!Speak a language other than English' : 'bilingual'}, \
           inplace = True)

In [ ]:
language

In [ ]:
other_language = language[['Geographic Area Name', 'id', 'bilingual']]

In [ ]:
other_language

In [ ]:
other_language = other_language.rename\
(columns = {"Geographic Area Name" : "geo_area"}, inplace = False)

In [ ]:
split = other_language["geo_area"].str.split(" ", n=1, expand=True)

In [ ]:
other_language["ZIPCODE"] = split[1]

In [ ]:
other_language

In [ ]:
boston_zipcodes = boston_zipcodes.merge(other_language, left_on='ZIPCODE', \
                                        right_on='ZIPCODE', how='inner').copy()

In [ ]:
boston_zipcodes = boston_zipcodes.replace(np.nan, 0)

In [ ]:
#Plot the number of other language speakers per zip code

fig, ax = plt.subplots(figsize=(10, 10))
boston_zipcodes.plot(legend=True,column='bilingual', \
                     cmap = 'copper',ax=ax, legend_kwds={'label':'Number of Speakers'})\
.set_title('Population Speaking Language Other Than Englishe', size=20)
cx.add_basemap(ax, source=cx.providers.Stamen.TonerLite, \
               crs=boston_zipcodes.crs.to_string())
ax.set_axis_off()

In [ ]:
#Zip codes with the highest number of (at least) bilingual individuals

head_language = boston_zipcodes.sort_values(by='bilingual', ascending=False)
head_language.head()

In [ ]:
#Zip codes with the lowest number of foreign-language speakers

head_language.tail()

## 8. School Enrollment

Young families typically desire, not only school enrollemnt for their own children, but to live around families who also value education. Below, the total population enrolled in school (kindergarten through 12th grade) is included in the overall suitability analysis.

In [ ]:
ACS = pd.read_csv('data/demographics/ACS.csv', header=1)

In [ ]:
ACS

In [ ]:
ACS.rename(columns = {'Estimate!!Total!!Population 3 years and over enrolled in school!!Kindergarten to 12th grade!!Kindergarten' : 'kinder_enrolled'}, \
           inplace = True)

In [ ]:
ACS_kinder = ACS[['Geographic Area Name', 'id', 'kinder_enrolled']]

In [ ]:
ACS_kinder

In [ ]:
ACS_kinder = ACS_kinder.rename\
(columns = {"Geographic Area Name" : "geo_area"}, inplace = False)

In [ ]:
split = ACS_kinder["geo_area"].str.split(" ", n=1, expand=True)

In [ ]:
ACS_kinder["ZIPCODE"] = split[1]

In [ ]:
ACS_kinder

In [ ]:
boston_zipcodes = boston_zipcodes.merge(ACS_kinder, left_on='ZIPCODE', \
                                        right_on='ZIPCODE', how='inner').copy()

In [ ]:
boston_zipcodes

In [ ]:
boston_zipcodes = boston_zipcodes.replace(np.nan, 0)

In [ ]:
#Plot the number of students enrolled per zip code

boston_zipcodes = boston_zipcodes.replace(np.nan, 0)
fig, ax = plt.subplots(figsize=(10, 10))
boston_zipcodes.plot(legend=True,column='kinder_enrolled', \
                     cmap = 'PuBuGn',ax=ax, legend_kwds={'label':'Total Number of School Enrollment'})\
.set_title('Boston Area School Enrollment by Zip Code', size=20)
cx.add_basemap(ax, source=cx.providers.Stamen.TonerLite, \
               crs=boston_zipcodes.crs.to_string())
ax.set_axis_off()

In [ ]:
head_enrolled = boston_zipcodes.sort_values(by='kinder_enrolled', ascending=False)
head_enrolled.head()

The zip codes just north and south of Boston have the highest school enrollment.

In [ ]:
tail_enrolled = boston_zipcodes.sort_values(by='kinder_enrolled', ascending=False)
head_enrolled.tail()

The five zipcodes with the lowest enrollment are 02047, 02199, 01901, 01467, and 02457.

## 9. Other Young Children

As a new parent, it is important to find connections with other young families to navigate raising children. Zip codes with high concentrations of young children likely contain daycares, playgroups, and other communities for young families.

In [ ]:
children = pd.read_csv('data/age_sex/age_sex.csv', header=1)

In [ ]:
children

In [ ]:
children.rename(columns = {'Estimate!!Total!!Total population!!AGE!!Under 5 years' : 'under_five'}, \
           inplace = True)

In [ ]:
children = children[['Geographic Area Name', 'id', 'under_five']]

In [ ]:
children = children.rename\
(columns = {"Geographic Area Name" : "geo_area"}, inplace = False)

In [ ]:
split = children["geo_area"].str.split(" ", n=1, expand=True)
children["ZIPCODE"] = split[1]

In [ ]:
boston_zipcodes = boston_zipcodes.replace(np.nan, 0)

In [ ]:
boston_zipcodes = boston_zipcodes.merge(children, left_on='ZIPCODE', \
                                        right_on='ZIPCODE', how='inner').copy()

In [ ]:
# Plot the number of young children per zipcode

fig, ax = plt.subplots(figsize=(10, 10))
boston_zipcodes.plot(legend=True, column='under_five', \
                     cmap='plasma', ax=ax, legend_kwds={'label':'Children Under Five'})\
.set_title('Total Population Under Five', size=20)
cx.add_basemap(ax, source=cx.providers.Stamen.TonerLite, \
               crs=boston_zipcodes.crs.to_string())
ax.set_axis_off()

In [ ]:
# Zip codes with the highest number of young children

head_children = boston_zipcodes.sort_values(by='under_five', ascending=False)
head_children.head()

In [ ]:
# Zip codes with the lowest number of young children

head_children.tail()

## 9. Reclassification 

Certain zipcodes will be more or less desirable for young families who consider all the above factors. Therefore, each zipcode should be reclassifed into three groups (highest, medium, and lowest desirability) based on the availability of resources and concentration of desirable demographics. For this analysis, quantile statistics were used to create suitability categories. For example, zip codes with a school count distribution higher than 50% will be categorized as highly desirable. These categorizations are used for each factor mentioned above.

In [ ]:
boston_zipcodes

In [ ]:
boston_zipcodes.describe()

In [ ]:
boston_zipcodes['reclass_enroll'] = np.NaN

In [ ]:
# View lowest quantile

boston_zipcodes.kinder_enrolled < boston_zipcodes.kinder_enrolled.quantile(0.25) 

In [ ]:
# Categorize school enrollment, 1-3

boston_zipcodes.loc[boston_zipcodes.kinder_enrolled <= boston_zipcodes.kinder_enrolled.quantile(0.25), 'reclass_enroll'] = 1
boston_zipcodes.loc[(boston_zipcodes.kinder_enrolled > boston_zipcodes.kinder_enrolled.quantile(0.25)) \
                    & (boston_zipcodes.kinder_enrolled <= boston_zipcodes.kinder_enrolled.quantile(0.5)),\
                    'reclass_enroll'] = 2
boston_zipcodes.loc[(boston_zipcodes.kinder_enrolled > boston_zipcodes.kinder_enrolled.quantile(0.5)) \
                    & (boston_zipcodes.kinder_enrolled <= boston_zipcodes.kinder_enrolled.quantile(1)),\
                    'reclass_enroll']= 3

In [ ]:
boston_zipcodes

In [ ]:
boston_zipcodes['reclass_schools'] = np.NaN

In [ ]:
# Categorize number of schools, 1-3

boston_zipcodes.loc[boston_zipcodes.school_count <= \
                    boston_zipcodes.school_count.quantile(0.25), 'reclass_schools'] = 1
boston_zipcodes.loc[(boston_zipcodes.school_count > boston_zipcodes.school_count.quantile(0.25))\
                    & (boston_zipcodes.school_count <= boston_zipcodes.school_count.quantile(0.5)),\
                    'reclass_schools'] = 2
boston_zipcodes.loc[(boston_zipcodes.school_count > boston_zipcodes.school_count.quantile(0.5))\
                    & (boston_zipcodes.school_count <= boston_zipcodes.school_count.quantile(1)),\
                    'reclass_schools']= 3

In [ ]:
boston_zipcodes['reclass_libs'] = np.NaN

In [ ]:
# Categorize number of libraries, 1-3

boston_zipcodes.loc[boston_zipcodes.lib_count <=\
                    boston_zipcodes.lib_count.quantile(0.25), 'reclass_libs'] = 1
boston_zipcodes.loc[(boston_zipcodes.lib_count > \
                     boston_zipcodes.lib_count.quantile(0.25)) \
                    & (boston_zipcodes.lib_count <= boston_zipcodes.lib_count.quantile(0.5)),\
                    'reclass_libs'] = 2
boston_zipcodes.loc[(boston_zipcodes.lib_count > boston_zipcodes.lib_count.quantile(0.5))\
                    & (boston_zipcodes.lib_count <= boston_zipcodes.lib_count.quantile(1)),\
                    'reclass_libs']= 3

In [ ]:
boston_zipcodes['reclass_markets'] = np.NaN

In [ ]:
# Categorize number of farmers markets, 1-3

boston_zipcodes.loc[boston_zipcodes.market_count <=\
                    boston_zipcodes.market_count.quantile(0.25), 'reclass_markets'] = 1
boston_zipcodes.loc[(boston_zipcodes.market_count > boston_zipcodes.market_count.quantile(0.25))\
                    & (boston_zipcodes.market_count <= boston_zipcodes.market_count.quantile(0.5)),\
                    'reclass_markets'] = 2
boston_zipcodes.loc[(boston_zipcodes.market_count > boston_zipcodes.market_count.quantile(0.5))\
                    & (boston_zipcodes.market_count <= boston_zipcodes.market_count.quantile(1)),\
                    'reclass_markets']= 3

In [ ]:
boston_zipcodes['reclass_language'] = np.NaN

In [ ]:
# Categorize foreign-language presence, 1-3

boston_zipcodes.loc[boston_zipcodes.bilingual <=\
                    boston_zipcodes.bilingual.quantile(0.25), 'reclass_language'] = 1
boston_zipcodes.loc[(boston_zipcodes.bilingual > boston_zipcodes.bilingual.quantile(0.25))\
                    & (boston_zipcodes.bilingual <= boston_zipcodes.bilingual.quantile(0.5)),\
                    'reclass_language'] = 2
boston_zipcodes.loc[(boston_zipcodes.bilingual > boston_zipcodes.bilingual.quantile(0.5))\
                    & (boston_zipcodes.bilingual <= boston_zipcodes.bilingual.quantile(1)),\
                    'reclass_language']= 3

In [ ]:
boston_zipcodes['reclass_children'] = np.NaN

In [ ]:
# Reclassify presence of other young families, 1-3

boston_zipcodes.loc[boston_zipcodes.under_five <=\
                    boston_zipcodes.under_five.quantile(0.25), 'reclass_children'] = 1
boston_zipcodes.loc[(boston_zipcodes.under_five > boston_zipcodes.under_five.quantile(0.25))\
                    & (boston_zipcodes.under_five <= boston_zipcodes.under_five.quantile(0.5)),\
                    'reclass_children'] = 2
boston_zipcodes.loc[(boston_zipcodes.under_five > boston_zipcodes.under_five.quantile(0.5))\
                    & (boston_zipcodes.under_five <= boston_zipcodes.under_five.quantile(1)),\
                    'reclass_children']= 3

In [ ]:
boston_zipcodes

Now, the Boston Zipcodes table includes a suitability rating (1-3) for each factor.

## 10. Weighted Suitability

Certain factors in this analysis should hold more weight than others. Proximity to schools and overall student enrollment, for example, would likely be considered more important than the number of nearby farmers markets. To analyze these weights, the factors included in these analysis will be weighted by importance.

In [ ]:
# Add weight to each variable 
boston_zipcodes['suitability'] = boston_zipcodes.reclass_enroll * .20 +\
boston_zipcodes.reclass_schools * .25 +\
boston_zipcodes.reclass_libs * .10 +\
boston_zipcodes.reclass_markets * .05 +\
boston_zipcodes.reclass_language * .25 +\
boston_zipcodes.reclass_children * .15

In [ ]:
boston_zipcodes

# Final Map

In [ ]:
#Plot map of final suitability 

fig, ax = plt.subplots(figsize=(10, 10))
boston_zipcodes.plot(legend=True, column='suitability', \
                     cmap='BuGn', ax=ax, legend_kwds={'label':'Suitability Score'}).set_title\
('Most Suitable Boston Zip Codes for Young Families', size=20)
cx.add_basemap(ax, source=cx.providers.Stamen.TonerLite, crs=boston_zipcodes.crs.to_string())
ax.set_axis_off()

According to this analysis, the below zip codes are considered **most** suitable for young families moving to the Boston area

In [ ]:
head_suitable = boston_zipcodes.sort_values(by='suitability', ascending=False)
head_suitable.head()

The following zip codes are considered **least** desirable for young Boston families

In [ ]:
head_suitable = boston_zipcodes.sort_values(by='suitability', ascending=False)
head_suitable.tail()